# Query Optimization

This lab will demonstrate ways to tune the performance of a database application by optimizing SQL queries.

## Pre-Requisites
1. Load the Account Dataset from `initialize_database.ipynb`

## Avoid using SELECT *
`SELECT *` returns all columns in a table that meet the constraints of the `SELECT` statement.

* Using `SELECT *` makes the response columns opaque; A change in database schema will change the results of the `SELECT`. This makes it harder to reason and profile the database.
* `SELECT *` often retrieves more data than needed, causing higher traffic load and adding latency.

Instead, specify the columns required explicitly.

In [1]:
### AVOID SELECT *
from LabDatabaseHandler import LabDatabaseHandler
db = LabDatabaseHandler()

db.reset_logs()

repeat = 10
sql = 'select * from account_dataset'
print(f"AVG Time taken for SELECT *: {db.execute(sql, repeat=repeat).print_duration()}")
sql = 'select email from account_dataset'
print(f"AVG Time taken for targeted SELECT: {db.execute(sql, repeat=repeat).print_duration()}")

db.generate_db_report()

AVG Time taken for SELECT *: 41.5ms
AVG Time taken for targeted SELECT: 21.7ms


[========================>] Parsed 3559 bytes of 3559 (100.00%), queries: 48, events: 0
LOG: Ok, generating html report...


### Avoid using SELECT DISTINCT

`SELECT DISTINCT` returns unique rows in a table that meet the constraints of the `SELECT` statement. However, it is expensive operation.

* `SELECT DISTINCT` is used 